# Creating custom graph types

Why:
- Specialized knowledge of the performed operations
- Need to convert back and forth from another data structure
- Existing structure that could behave as a graph

Two broad categories:
- Generic types: data structure representing most graphs
- Specialized types: represent a particular shape

## Matrix-based graph

In [1]:
using LightGraphs
using GraphPlot
using SparseArrays
const LG = LightGraphs

LightGraphs

In [28]:
"""
    MatrixGraph{D, M <: AbstractMatrix{Bool}}

A matrix-based graph type.
The boolean matrix `m` is the adjacency matrix.
`D` is a boolean type parameter indicating directedness.
"""
struct MatrixGraph{D, M <: AbstractMatrix{Bool}} <: AbstractGraph{Int}
    m::M
end

function MatrixGraph{D}(m::M) where {D, M <: AbstractMatrix{Bool}}
    return MatrixGraph{D, M}(m)
end

function MatrixGraph{D}(nvtx::Integer) where {D}
    m = spzeros(Bool, nvtx, nvtx)
    return MatrixGraph{D, typeof(m)}(m)
end

## Implementing the interface

In [32]:
LG.is_directed(::MatrixGraph{D}) where {D} = D
LG.edgetype(::MatrixGraph) = LG.SimpleGraphs.SimpleEdge{Int}
LG.nv(g::MatrixGraph) = size(g.m, 1)

LG.ne(g::MatrixGraph{false}) = sum(g.m) ÷ 2
LG.ne(g::MatrixGraph{true}) = sum(g.m)

LG.vertices(g::MatrixGraph) = Base.OneTo(nv(g))

In [18]:
g = MatrixGraph{false}(4)

MatrixGraph{false,SparseMatrixCSC{Bool,Int64}}(4×4 SparseMatrixCSC{Bool,Int64} with 0 stored entries)

In [17]:
# edges returns an iterator over edges
# depends on directedness

function LG.edges(g::MatrixGraph{true})
    n = nv(g)
    return (LightGraphs.SimpleGraphs.SimpleEdge(i,j) for i in 1:n for j in 1:n if g.m[i,j])
end

function LG.edges(g::MatrixGraph{false})
    n = nv(g)
    return (LightGraphs.SimpleGraphs.SimpleEdge(i, j) for i in 1:n for j in i:n if g.m[i,j])
end

In [22]:
LG.outneighbors(g::MatrixGraph, v) = [j for j in vertices(g) if g.m[v, j]]

LG.inneighbors(g::MatrixGraph,  v) = [i for i in vertices(g) if g.m[i, v]]

LG.has_vertex(g::MatrixGraph, v) = v in vertices(g)
LG.has_edge(g::MatrixGraph, i, j) = g.m[i,j]

## Mutating our graph

In [24]:
function LG.add_edge!(g::MatrixGraph{true}, i, j)
    has_edge(g, i, j) && return false
    has_vertex(g, i) && has_vertex(g, j) || return false
    g.m[i, j] = true
end

function LG.add_edge!(g::MatrixGraph{false}, i, j)
    has_edge(g, i, j) && return false
    has_vertex(g, i) && has_vertex(g, j) || return false
    g.m[i, j] = g.m[j, i] = true
end

function LG.rem_edge!(g::MatrixGraph{true}, i, j)
    has_edge(g, i, j) || return false
    g.m[i, j] = false
    return true
end

function LG.rem_edge!(g::MatrixGraph{false}, i, j)
    has_edge(g, i, j) || return false
    g.m[i, j] = g.m[j, i] = false
    return true
end

Note: we do not implement vertex removal or addition.  
$\Rightarrow$ Requires replacing the matrix.

## Specializing non-mandatory

In [30]:
using BenchmarkTools: @btime

g = MatrixGraph{true}(sprand(Bool, 1000, 1000, 0.2));

In [37]:
@btime adjacency_matrix($g)
# why so long?
LG.adjacency_matrix(g::MatrixGraph) = Int.(g.m)
@btime adjacency_matrix($g);
# that's better

## Further resources

Matrix-based graphs:  
https://github.com/nassarhuda/MatrixNetworks.jl